# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime,timedelta
import requests
import json

# Defining paths

In [197]:
dict_of_data = {'Agrinio':'Agrinio_sample.csv',
                'Alexandroupoli':'Alexandroupoli_sample.csv',
                'Arta':'Arta_sample.csv',
                'Athens':'Athens_sample.csv',
                'Chalcis':'Chalcis_sample.csv',
                'Corinth':'Corinth_sample.csv',
                'Ioannina':'Ioannina_sample.csv',
                'Kalamata':'Kalamata_sample.csv',
                'Kozani':'Kozani_sample.csv',
                'Lamia':'Lamia_sample.csv',
                'Larissa':'Larissa_sample.csv',
                'Nafplion':'Nafplion_sample.csv',
                'Patras':'Patras_sample.csv',
                'Volos':'Volos_sample.csv',
                'Thessaloniki':'Thessaloniki_sample.csv'}

In [199]:
dict_datas = {name:pd.read_csv(dict_of_data[name]) for name in list(dict_of_data.keys())}
dict_city_lat_lon = {name:[dict_datas[name].lat[0],dict_datas[name].lon[0]] for name in list(dict_datas.keys())}

# Defining useful functions

In [203]:
def get_total_info_from_dictionary(aDictionary):
    returned_dictionary = {}
    
    main_info    = aDictionary['main']
    wind_info    = aDictionary['wind']
    cloud_info   = aDictionary['clouds']
    weather_info = aDictionary['weather'][0]
    
    certain_dt   = aDictionary['dt'] 
    date         = datetime.utcfromtimestamp(certain_dt)  
    date_gr      = datetime.fromtimestamp(certain_dt)      
    
    returned_dictionary['dt']                    = certain_dt
    returned_dictionary['dt_iso']                = str(date.strftime('%Y-%m-%d %H:%M:%S'))+' +0000 UTC'
    returned_dictionary['timezone']              = int((date_gr - date ).total_seconds())
    returned_dictionary['temp']                  = main_info['temp']
    returned_dictionary['feels_like']            = main_info['feels_like']
    returned_dictionary['temp_min']              = main_info['temp_min']
    returned_dictionary['temp_max']              = main_info['temp_max']
    returned_dictionary['pressure']              = main_info['pressure']
    returned_dictionary['humidity']              = main_info['humidity']
    returned_dictionary['wind_speed']            = wind_info['speed']
    returned_dictionary['wind_deg']              = wind_info['deg']
    returned_dictionary['clouds_all']            = cloud_info['all']
    returned_dictionary['weather_id']            = weather_info['id']
    returned_dictionary['weather_main']          = weather_info['main']
    returned_dictionary['weather_description']   = weather_info['description']
    returned_dictionary['weather_icon']          = weather_info['icon']
    
    return returned_dictionary

def collect_total_info(aList):
    list_of_dt                  = []
    list_of_dt_iso              = []
    list_of_timezone            = []
    list_of_temp                = []
    list_of_feels_like          = []
    list_of_temp_min            = []
    list_of_temp_max            = []
    list_of_pressure            = []
    list_of_humidity            = []
    list_of_wind_speed          = []
    list_of_wind_deg            = []
    list_of_clouds_all          = []
    list_of_weather_id          = []
    list_of_weather_main        = []
    list_of_weather_description = []
    list_of_weather_icon        = []
    for aDictionary in aList:
        partial_info = get_total_info_from_dictionary(aDictionary)
        list_of_dt.append(partial_info['dt'])
        list_of_dt_iso.append(partial_info['dt_iso'])
        list_of_timezone.append(partial_info['timezone'])
        list_of_temp.append(  partial_info['temp']  - 273.15 )           # Transform Kelvin to Celcius 
        list_of_feels_like.append(partial_info['feels_like'] - 273.15 )  # Transform Kelvin to Celcius
        list_of_temp_min.append(partial_info['temp_min'] - 273.15 )      # Transform Kelvin to Celcius
        list_of_temp_max.append(partial_info['temp_max'] - 273.15)       # Transform Kelvin to Celcius
        list_of_pressure.append(partial_info['pressure'])
        list_of_humidity.append(partial_info['humidity'])
        list_of_wind_speed.append(partial_info['wind_speed'])
        list_of_wind_deg.append(partial_info['wind_deg' ] )
        list_of_clouds_all.append(partial_info['clouds_all'])
        list_of_weather_id.append(partial_info['weather_id'])
        list_of_weather_main.append(partial_info['weather_main'])
        list_of_weather_description.append(partial_info['weather_description'])
        list_of_weather_icon.append(partial_info['weather_icon'])
    
    return [list_of_dt,list_of_dt_iso,list_of_timezone,
            list_of_temp, list_of_feels_like, list_of_temp_min,
            list_of_temp_max, list_of_pressure, list_of_humidity,
            list_of_wind_speed, list_of_wind_deg, list_of_clouds_all,
            list_of_weather_id, list_of_weather_main,list_of_weather_description,
            list_of_weather_icon]    

# Load missing data

In [219]:
final_dictionary = {}

dateTime = datetime.today()
unixtime_today  = int(time.mktime(dateTime.timetuple()))                     # Calculate current unixtime
unixtime_latest = int(dict_datas[list(dict_datas.keys())[0]].dt.values[-1])  # Calculate latest data's unixtime
gap_days        = (unixtime_today - unixtime_latest) / (3600 * 24)           # Calculate missing days
number_of_iterations = int(gap_days / 7 )                                    # Calculate the number of API calls needed ( maximum days per call is 7 )
for i in range(0,number_of_iterations + 1):
    print('Iteration ' + str(i) + ' from ' + str(number_of_iterations))
    for name in dict_city_lat_lon.keys():
        unixtime_start = dict_datas[name].dt.values[-1] + 3600
        unixtime_end   = unixtime_start + 3600 * (7 * 24 )
        lat = dict_city_lat_lon[name][0]
        lon = dict_city_lat_lon[name][1]
        my_key = ''                                                          # Put yor API key here
        url = 'http://history.openweathermap.org/data/2.5/history/city?lat='+str(lat)+'&lon='+str(lon)+'&type=hour&start='+str(unixtime_start)+'&end='+str(unixtime_end)+'&appid='+my_key
        current_request = requests.get(url)
        print(current_request.status_code)
        final_dictionary[name] = current_request.json()
        temp_dat = collect_total_info(final_dictionary[name]['list'])
        list_of_city_name = [name] * len(temp_dat[0])
        list_of_lat       = [lat]  * len(temp_dat[0])
        list_of_lon       = [lon]  * len(temp_dat[0])
        list_of_nan       = [np.NaN] * len(temp_dat[0])

        extra_df = pd.DataFrame({'dt':temp_dat[0],'dt_iso':temp_dat[1],'timezone':temp_dat[2],
                                 'city_name':list_of_city_name, 'lat':list_of_lat, 'lon':list_of_lon,
                                 'temp':temp_dat[3], 'feels_like':temp_dat[4], 'temp_min':temp_dat[5],
                                 'temp_max':temp_dat[6], 'pressure':temp_dat[7], 'sea_level':list_of_nan,
                                 'grnd_level':list_of_nan, 'humidity':temp_dat[8], 'wind_speed':temp_dat[9],
                                 'wind_deg':temp_dat[10], 'rain_1h':list_of_nan, 'rain_3h':list_of_nan,
                                 'snow_1h':list_of_nan, 'snow_3h':list_of_nan, 'clouds_all':temp_dat[11],
                                 'weather_id':temp_dat[12], 'weather_main':temp_dat[13], 'weather_description':temp_dat[14],
                                 'weather_icon':temp_dat[15] }
                               )
        dict_datas[name] = dict_datas[name].append(extra_df, ignore_index=True)

Iteration 0 from 0
200


# Save updated data

In [209]:
for name in dict_datas.keys():
    dict_datas[name].to_csv(name+'.csv',index=False)